<a href="https://colab.research.google.com/github/thfruchart/tnsi-2020/blob/master/Chap14/COURS_AVEC_TESTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amélioration de la recherche : l'algorithme de Boyer-Moore

[Principe de l'algorithme et premier exemple](https://pixees.fr/informatiquelycee/n_site/nsi_term_algo_boyer.html)

             0            i    i+j             len(t)
        t   |............| | | |c| | | .......| 
                        m| | | |x| | | 
                          0     j     len(m)

* on recherche un motif `m` dans un texte `t`
* on va comparer les caractères de `m` avec ceux de `t` avec deux boucles imbriquées
   * cela signifie qu'on va tester si `m[j]==t[i+j]` 
   * avec  `j` indice du motif `m` donc `0<=j<len(m)`
   * et  `i+j` indice du texte `t` donc `0<=i+j<len(t)`
* on commence par tester le **dernier caractère** du motif, et on "remonte" vers le premier caractère du motif : `for j in range(len(m)-1,-1,-1)`
   * si aucune différence n'existe, alors le motif apparaît dans le texte
   * si au moins une différence existe, alors le motif n'apparaît **pas à cet endroit** du texte.
   * on peut "décaler" le motif dans le texte pour démarrer une nouvelle recherche. 
   * par défaut, le décalage est `k=1`, mais ce décalage peut être plus grand dans certains cas 
   * on incrémente `i` avant le prochain "tour de boucle" :   `i = i+k`

## Prétraitement du motif

### Exemple

    motif   : a b r a c a d a b r a
    indices   0 1 2 3 4 5 6 7 8 9 10

Tableau des "indices précédents" pour chaque valeur k et chaque caractère du motif

| caractère|0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|
|:--------:|:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
|        a |  | 0| 0|0 | 3|3 |5 |5 |7 |7 |7 |  
|        b |  |  | | | | | | | | | |
|        r |  |  |  |2 |2 |2 |2 |2 |2 |2 |9 |
|        c |  |  |  |  |  |4 |4 |4 |4 |4 |4 |
|        d |  |  |  |  |  |  |  |7 |7 |7 |7 |


### Travail à faire

1. COMPLÉTER la  ligne incomplète du tableau ci-dessus
2. Compléter le code de la cellule suivante pour définir la fonction `table_boyer_moore` qui implémente le tableau des indices "de Boyer-Moore" sous forme d'une liste de dictionnaires. 
3. Tester votre code avec les assertions données en dessous.

In [2]:
def table_boyer_moore(m):
    """construit la table des indices de Boyer-Moore dans une liste de dictionnaires
    dico_list[j][c] est le plus grand indice du caractère c dans m[0:j] s'il existe
    et n'est pas défini sinon"""
    dico_list = [ {} for j in range(len(m))]

    for j in range(len(m)):
        for k in range(j):
            c = m[k]
            dico_list[j][c]= k  # ligne à compléter
    return dico_list
    

In [3]:
tbm = table_boyer_moore('abracadabra')
assert tbm[1]['a']==0
assert tbm[4]['a']==3
assert tbm[6]['a']==5
assert tbm[8]['a']==7
assert tbm[5]['b']==1
assert tbm[9]['b']==8
assert tbm[5]['c']==4
assert tbm[7]['d']==6
assert tbm[10]['a']==7
assert 'a' not in tbm[0]
assert 'b' not in tbm[1]
assert 'r' not in tbm[2]
assert 'c' not in tbm[4]
assert 'd' not in tbm[6]
print('test ok')

test ok


### Calcul du décalage


*  Dans le cas où le texte contient  le caractère `'b'`  en face de `m[5]`,  vérifier que le **décalage** à opérer vaut **4**. 
*  Donner la valeur du décalage à opérer lorsque le texte contient :
   1.  `'r'` en face de `m[5]`  : 3
   2.  `'c'` en face de `m[5]`  : 1
   3.  `'d'` en face de `m[5]`  : 6

    texte   ....................b......................
                                *
            motif   : a b r a c a d a b r a
            indices   0 1 2 3 4 5 6 7 8 9 10

Tableau des "indices précédents" pour chaque valeur k et chaque caractère du motif

| caractère|0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|
|:--------:|:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
|        a |  | 0| 0|0 | 3|3 |5 |5 |7 |7 |7 |  
|        b |  |  |1 |1 |1 |1 |1 |1 |1 |8 |8 |
|        r |  |  |  |2 |2 |2 |2 |2 |2 |2 |9 |
|        c |  |  |  |  |  |4 |4 |4 |4 |4 |4 |
|        d |  |  |  |  |  |  |  |7 |7 |7 |7 |


In [4]:
def decalage(table,j,c):
    """Renvoie le décalage à effectuer lorsque le caractère c 
    est différent du caractère d'indice j dans le motif recherché.
    Le décalage est obtenu avec la table des indices de Boyer-Moore"""
    if c in table[j]:
        return j - table[j][c]
    else :
        return j+1

# Fonction recherche

In [ ]:
def recherche(m, t):
    """affiche tous les indices des occurrences du motif m dans le texte t
    avec l'algorithme de Boyer-Moore"""
    table = table_boyer_moore(m)
    i = 0 #premier indice du texte t
    while i <= len(t)-len(m):
        k = 0 #le décalage est initialisé à 0
        for j in range(len(m)-1,-1,-1): #parcours depuis la fin du motif
            if t[i+j] != m[j]:
                k = decalage(table, j, t[i+j])
                break
        # fin de la boucle for
        if k==0 :
            print('occurrence à la position',i)
            k = 1
        i = i+k

## différents cas à tester


0. le motif n'apparaît aucune fois

In [ ]:
recherche('BBB', 'ATGTCACGTATCGACGTAG')

1. le motif apparaît une fois

In [ ]:
recherche('ATCG', 'ATGTCACGTATCGACGTAG')

occurrence à la position 9


2. Le motif apparaît plusieurs fois

In [ ]:
recherche('ATCG', 'ATGTCACGTATCGACGTATCGA')

occurrence à la position 9
occurrence à la position 17


3. Le motif apparaît en DÉBUT de texte

In [ ]:
recherche('ATGT', 'ATGTCACGTATCGACGTATCGAG')

occurrence à la position 0


4. Le motif apparaît en FIN de texte
**PROBLÈME** => corrigé avec `while i <= len(t)-len(m)`

In [ ]:
recherche('TCGAG', 'ATGTCACGTATCGACGTATCGAG')

occurrence à la position 18


#### Travail à faire : tester cette fonction
Attention ... il y a une "petite" erreur à corriger

# Version 2 de la fonction recherche

In [1]:
def recherche(m, t):
    """RENVOIE une liste avec tous les indices des occurrences 
    du motif m dans le texte t avec l'algorithme de Boyer-Moore"""
    resu = [] #résultat initialement vide
    table = table_boyer_moore(m)
    i = 0 #premier indice du texte t
    while i <= len(t)-len(m):
        k = 0 #le décalage est initialisé à 0
        for j in range(len(m)-1,-1,-1): #parcours depuis la fin du motif
            if t[i+j] != m[j]:
                k = decalage(table, j, t[i+j])
                break # met fin à la boucle FOR
        # fin de la boucle for
        if k==0 :
            resu.append(i)
            k = 1
        i = i+k
    return resu

In [10]:
assert recherche('BBB', 'ATGTCACGTATCGACGTAG')==[]
assert recherche('ATGT', 'ATGTCACGTATCGACGTAG')==[0]
assert recherche('CGTAG', 'ATGTCACGTATCGACGTAG')==[14]
assert recherche('TCA', 'ATGTCACGTATCTCAGACGTTCAAG')==[3,12,20]
assert recherche ('AAAAA', 'AAA')==[]
print('test OK')


test OK
